In [ ]:
import numpy as np
# from sympy import primerange # primerange を使う場合
# from sklearn.decomposition import PCA # UMAPのコードにあったが、ここでは不要
# import umap.umap_ as umap # UMAPのコードにあったが、ここでは不要

# ===== データ読み込み =====
x_data = np.load("x_data.npy")
y_data = np.load("y_data.npy")

# ===== 補助特徴量の作成 =====
num_points = len(x_data)

# Option 1: primerange を使用 (計算時間に注意)
# primes = list(primerange(1, 1500000000))[:num_points]
# log_primes = np.log(primes)
# kappa_list = 1 - (np.log(np.log(primes)) / np.log(primes))

# Option 2: 保存されたファイルを使用 (推奨)
log_primes = np.load("log_primes.npy")[:num_points] # num_points に合わせてスライス
kappa_list = np.load("kappa_list.npy")[:num_points] # num_points に合わせてスライス

# ===== 特徴行列を作成（4次元）=====
X_original = np.stack([x_data, y_data, kappa_list, log_primes], axis=1)
print(f"元の高次元データ X_original の形状: {X_original.shape}")

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# t-SNEのパラメータ設定
n_components_tsne = 2
perplexity_tsne = 30.0
learning_rate_tsne = 'auto'
n_iter_tsne = 1000
init_tsne = 'pca'

tsne = TSNE(n_components=n_components_tsne,
            perplexity=perplexity_tsne,
            learning_rate=learning_rate_tsne,
            n_iter=n_iter_tsne,
            init=init_tsne,
            random_state=42)

print("t-SNE fitting started...")
X_tsne = tsne.fit_transform(X_original)
print("t-SNE fitting finished.")

In [ ]:
# ✅ Colab用日本語フォント設定（必須）
!apt-get -y install fonts-noto-cjk
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams["font.family"] = font_prop.get_name()

# ----- 可視化の準備 -----
# UMAP可視化で使用した他の情報もロード (X_original の行数と一致させる)
# num_points = X_original.shape[0] # ステップ1で定義済み

# Colab用日本語フォント設定 (もし未実行なら実行)
# !apt-get -y install fonts-noto-cjk
# import matplotlib.font_manager as fm
# font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
# font_prop = fm.FontProperties(fname=font_path)
# plt.rcParams["font.family"] = font_prop.get_name()


# 準励起点（上位10%）の定義は前回と同様 (y_data を info_potential とみなす場合)
current_info_potential_for_tsne = X_original[:, 1] # y_data
threshold_tsne = np.percentile(current_info_potential_for_tsne, 90)
is_excited_tsne = current_info_potential_for_tsne >= threshold_tsne

# true_return_point のインデックスを取得
# ご提示いただいた `true_return_point.npy` 生成コードから必要な変数をロード
umap_embedding_for_true_point_logic = np.load("umap_embedding.npy") # UMAP埋め込み
stream_vectors_for_true_point_logic = np.load("stream_vectors.npy") # ストリームベクトル
spike_indices_for_true_point_logic = np.load("spike_indices.npy")   # スパイク点の元のインデックス
stream_end_cluster_labels_for_true_point_logic = np.load("stream_end_cluster_labels.npy") # 終点クラスタラベル

valid_mask_for_true_point_logic = stream_end_cluster_labels_for_true_point_logic != -1
start_indices_for_true_point_logic = spike_indices_for_true_point_logic[valid_mask_for_true_point_logic]
start_points_umap_for_true_point_logic = umap_embedding_for_true_point_logic[start_indices_for_true_point_logic]
stream_vectors_2d_for_true_point_logic = stream_vectors_for_true_point_logic[valid_mask_for_true_point_logic, :2]
end_points_umap_for_true_point_logic = start_points_umap_for_true_point_logic + stream_vectors_2d_for_true_point_logic
distances_for_true_point_logic = np.linalg.norm(start_points_umap_for_true_point_logic - end_points_umap_for_true_point_logic, axis=1)
epsilon_for_true_point_logic = 1.5 # UMAP空間での距離の閾値
return_mask_for_true_point_logic = distances_for_true_point_logic < epsilon_for_true_point_logic

# これが元の高次元データ X_original における「#1点」のインデックス群
true_return_point_original_indices = start_indices_for_true_point_logic[return_mask_for_true_point_logic]

# t-SNE空間での「#1点」の座標を取得
true_return_points_tsne_coords = X_tsne[true_return_point_original_indices]


# --- データフレーム化 (t-SNE結果) ---
df_tsne = pd.DataFrame({
    "x": X_tsne[:, 0],
    "y": X_tsne[:, 1],
    "info_potential": current_info_potential_for_tsne,
    "is_excited": is_excited_tsne
})
# オプション: もしクラスタラベルも比較したい場合
# cluster_labels = np.load("cluster_labels.npy")[:num_points]
# df_tsne["cluster"] = cluster_labels


# --- 可視化 ---
plt.figure(figsize=(9, 7))
sns.scatterplot(data=df_tsne, x="x", y="y", hue="is_excited", palette={True: "red", False: "lightgray"}, s=30, zorder=1)

if len(true_return_points_tsne_coords) > 0:
    plt.scatter(true_return_points_tsne_coords[:, 0], true_return_points_tsne_coords[:, 1],
                color="blue", s=100, label=f"#1点（真）in t-SNE ({len(true_return_points_tsne_coords)}件)", marker="X", zorder=2, alpha=0.7)

plt.title("t-SNE: 準励起点（上位10%）と #1点 の幾何学的位置", fontproperties=font_prop)
plt.xlabel("t-SNE-1", fontproperties=font_prop)
plt.ylabel("t-SNE-2", fontproperties=font_prop)
if len(true_return_points_tsne_coords) > 0:
    plt.legend(prop=font_prop)
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import pandas as pd

# 前提：X_original, X_tsne, df_tsne が既に定義されているとします。
# df_tsne は X_tsne の座標と元のインデックス (X_original のインデックス) を持つDataFrame
# 例: df_tsne = pd.DataFrame(X_tsne, columns=['x', 'y'])
#     df_tsne['original_index'] = np.arange(len(X_original)) # もしdf_tsneに元のindexがなければ

# --- 候補領域① ---
x_min1, x_max1 = -110, -90
y_min1, y_max1 = 0, 25 # ユーザー提案の0-20より少し広げてみました

mask1 = (df_tsne['x'] >= x_min1) & (df_tsne['x'] <= x_max1) & \
        (df_tsne['y'] >= y_min1) & (df_tsne['y'] <= y_max1)
indices_seq1_unsorted = df_tsne[mask1].index.to_numpy() # これが X_original のインデックスに対応
print(f"候補領域①で抽出された点数: {len(indices_seq1_unsorted)}")

# --- 候補領域② ---
x_min2, x_max2 = 70, 105
y_min2, y_max2 = -15, 15

mask2 = (df_tsne['x'] >= x_min2) & (df_tsne['x'] <= x_max2) & \
        (df_tsne['y'] >= y_min2) & (df_tsne['y'] <= y_max2)
indices_seq2_unsorted = df_tsne[mask2].index.to_numpy()
print(f"候補領域②で抽出された点数: {len(indices_seq2_unsorted)}")

# 抽出された点の数を確認し、少なすぎる/多すぎる場合は範囲を調整してください。
# 目安として、FFTにある程度の点数（数十～数百）があると安定しやすいです。

In [ ]:
# X_original がロード済みであること
# X_original の1列目が y_data (情報ポテンシャル) であると仮定

# --- 候補領域①のシーケンスをソート ---
if len(indices_seq1_unsorted) > 0:
    y_values_seq1 = X_original[indices_seq1_unsorted, 1] # 1列目がy_data
    sort_order_seq1 = np.argsort(y_values_seq1)
    ordered_indices_seq1 = indices_seq1_unsorted[sort_order_seq1]
    print(f"候補領域①のソート済みインデックス数: {len(ordered_indices_seq1)}")
else:
    ordered_indices_seq1 = np.array([])
    print("候補領域①に該当する点がありませんでした。範囲を見直してください。")

# --- 候補領域②のシーケンスをソート ---
if len(indices_seq2_unsorted) > 0:
    y_values_seq2 = X_original[indices_seq2_unsorted, 1] # 1列目がy_data
    sort_order_seq2 = np.argsort(y_values_seq2)
    ordered_indices_seq2 = indices_seq2_unsorted[sort_order_seq2]
    print(f"候補領域②のソート済みインデックス数: {len(ordered_indices_seq2)}")
else:
    ordered_indices_seq2 = np.array([])
    print("候補領域②に該当する点がありませんでした。範囲を見直してください。")

In [ ]:
# X_original と dc_dt.npy がロード済みであること
# dc_dt.npy は X_original と同じ順序・長さのスパイク点に対応していると仮定

# --- 候補領域①の三指標データ ---
if len(ordered_indices_seq1) > 0:
    info_seq1 = X_original[ordered_indices_seq1, 1]  # y_data (情報ポテンシャル)
    # dc_dt.npy のインデックス参照方法に注意：ordered_indices_seq1 が直接使えるか、
    # あるいは spike_indices の中での順番に変換する必要があるか確認
    # ここでは ordered_indices_seq1 が元のデータ全体に対するインデックスと仮定
    dc_seq1 = np.load("dc_dt.npy")[ordered_indices_seq1]
    kappa_seq1 = X_original[ordered_indices_seq1, 2] # kappa_list
    print(f"候補領域①の指標データ形状: info={info_seq1.shape}, dc={dc_seq1.shape}, kappa={kappa_seq1.shape}")

# --- 候補領域②の三指標データ ---
if len(ordered_indices_seq2) > 0:
    info_seq2 = X_original[ordered_indices_seq2, 1]  # y_data
    dc_seq2 = np.load("dc_dt.npy")[ordered_indices_seq2]
    kappa_seq2 = X_original[ordered_indices_seq2, 2] # kappa_list
    print(f"候補領域②の指標データ形状: info={info_seq2.shape}, dc={dc_seq2.shape}, kappa={kappa_seq2.shape}")

# 注意：dc_dt.npy の読み込みとインデックスの対応について
# もし X_original を作成する際に使った spike_indices がある場合、
# dc_dt.npy もその spike_indices を通して X_original の行に対応づける必要があります。
# 例：
# all_dc_dt = np.load("dc_dt.npy")
# dc_dt_for_X_original = all_dc_dt[spike_indices_used_for_X_original]
# # その後
# dc_seq1 = dc_dt_for_X_original[ordered_indices_seq1]
# この部分は元のデータ構造に合わせて調整してください。
# もし `X_original` 作成時に `dc_dt` も含めていたなら、`X_original` から直接列を指定して抽出できます。

In [ ]:
from scipy.fft import fft, fftfreq
import numpy as np
import matplotlib.pyplot as plt # プロット用

def analyze_fft_and_phase(signal_dict, sampling_interval=1, title_suffix=""):
    """
    複数の信号に対してFFT解析を行い、スペクトルと位相情報をプロット・表示する。
    signal_dict: {'label1': signal1, 'label2': signal2, ...} の形式の辞書
    sampling_interval: サンプリング間隔 (デフォルトは1)
    title_suffix: プロットのタイトルに追加する接尾辞
    """
    plt.figure(figsize=(12, 8))
    colors = ['blue', 'red', 'green', 'purple', 'orange'] # 必要に応じて追加
    peak_frequencies = {}
    phases_at_peak = {}

    # --- スペクトルプロット ---
    plt.subplot(2, 1, 1)
    for i, (label, signal) in enumerate(signal_dict.items()):
        if len(signal) < 2: # 点数が少なすぎるとFFTできない
            print(f"信号 '{label}' の点数が少なすぎるためFFTをスキップします。")
            continue
        N = len(signal)
        yf = fft(signal - np.mean(signal))  # DC成分除去
        xf = fftfreq(N, sampling_interval)[:N//2]

        # スペクトル強度
        magnitude = 2.0/N * np.abs(yf[:N//2])
        plt.plot(xf, magnitude, label=label, color=colors[i % len(colors)])

        # 卓越周波数と位相の特定 (DC成分0Hzを除く)
        if len(xf) > 1: # 0Hz以外の周波数成分がある場合
            peak_idx = np.argmax(magnitude[1:]) + 1 # 0Hzを除いた最大値のインデックス
            peak_freq = xf[peak_idx]
            phase_rad = np.angle(yf[peak_idx]) # 卓越周波数での位相 (ラジアン)
            phase_deg = np.degrees(phase_rad)  # 度に変換
            peak_frequencies[label] = peak_freq
            phases_at_peak[label] = phase_deg
            print(f"'{label}': 卓越周波数 = {peak_freq:.4f}, 位相 = {phase_deg:.2f}° (at index {peak_idx})")
        else:
            print(f"信号 '{label}' の周波数成分が少なすぎるため、卓越周波数・位相を特定できません。")


    plt.title(f"3指標のFFTスペクトル {title_suffix}", fontproperties=font_prop)
    plt.xlabel("周波数", fontproperties=font_prop)
    plt.ylabel("スペクトル強度", fontproperties=font_prop)
    plt.legend(prop=font_prop)
    plt.grid(True)

    # --- 位相情報の表示 (例としてテキストで) ---
    # より詳細な位相比較は別途行う（例：位相差の計算など）
    # plt.subplot(2, 1, 2) # 位相プロット用のスペース（今回はテキストで表示）
    # phase_text = "卓越周波数における位相:\n"
    # for label, freq in peak_frequencies.items():
    #     phase_text += f"  {label}: Freq={freq:.4f}, Phase={phases_at_peak[label]:.2f}°\n"
    # plt.text(0.05, 0.5, phase_text, fontsize=10, va='center')
    # plt.axis('off') # 軸を非表示

    plt.tight_layout()
    plt.show()

    return peak_frequencies, phases_at_peak

# --- 候補領域①のFFT解析 ---
if 'info_seq1' in locals() and len(info_seq1) > 1: # データが存在し、点数が十分か確認
    signals_seq1 = {
        "情報ポテンシャル": info_seq1,
        "dc/dt": dc_seq1,
        "κ(p)": kappa_seq1
    }
    print("\n--- 候補領域①のFFT解析結果 ---")
    peaks_seq1, phases_seq1 = analyze_fft_and_phase(signals_seq1, title_suffix="- 候補領域①")
    # 位相差の計算など、さらに詳細な比較
    if len(phases_seq1) == 3: # 3指標全ての位相が取れた場合
        # 例: info と dc の位相差
        # phase_diff_info_dc_s1 = phases_seq1["情報ポテンシャル"] - phases_seq1["dc/dt"]
        # print(f"候補領域① Info vs dc/dt 位相差: {phase_diff_info_dc_s1 % 360:.2f}°")
        pass # ここで位相差を計算・表示するロジックを追加

# --- 候補領域②のFFT解析 ---
if 'info_seq2' in locals() and len(info_seq2) > 1:
    signals_seq2 = {
        "情報ポテンシャル": info_seq2,
        "dc/dt": dc_seq2,
        "κ(p)": kappa_seq2
    }
    print("\n--- 候補領域②のFFT解析結果 ---")
    peaks_seq2, phases_seq2 = analyze_fft_and_phase(signals_seq2, title_suffix="- 候補領域②")
    if len(phases_seq2) == 3:
        pass # 同様に位相差を計算・表示

In [ ]:
# --- 前提: analyze_fft_and_phase_extended 関数が定義済みであること ---
# --- 前提: 候補領域①のデータ (info_seq1, dc_seq1, kappa_seq1) が定義済みであること ---
# --- 前提: 候補領域①のFFT結果 (peaks_seq1) が取得済みであること ---
# --- 前提: 候補領域②のデータ (info_seq2, dc_seq2, kappa_seq2) が定義済みであること ---
# --- 前提: 候補領域②のFFT結果 (peaks_seq2) が取得済みであること ---

# --- 候補領域①の dc/dt 卓越周波数での再解析 ---
if 'peaks_seq1' in locals() and 'dc/dt' in peaks_seq1 and \
   'info_seq1' in locals() and len(info_seq1) > 1:
    signals_seq1_for_target = {
        "情報ポテンシャル": info_seq1,
        "dc/dt": dc_seq1,
        "κ(p)": kappa_seq1
    }
    target_freqs_seq1 = {
        f"dc/dt (Seq1 peak at {peaks_seq1['dc/dt']:.4f})": peaks_seq1['dc/dt']
    }
    print("\n--- 候補領域① 再解析 (dc/dtの卓越周波数に着目) ---")
    _, _, _ = analyze_fft_and_phase_extended(
        signals_seq1_for_target,
        target_frequencies_dict=target_freqs_seq1,
        title_suffix="- 候補領域① (dc/dt Freq Focus)"
    )
else:
    print("候補領域①のデータまたはdc/dtの卓越周波数が未定義です。")

# --- 候補領域②の dc/dt 卓越周波数での再解析 ---
if 'peaks_seq2' in locals() and 'dc/dt' in peaks_seq2 and \
   'info_seq2' in locals() and len(info_seq2) > 1:
    signals_seq2_for_target = {
        "情報ポテンシャル": info_seq2,
        "dc/dt": dc_seq2,
        "κ(p)": kappa_seq2
    }
    target_freqs_seq2 = {
        f"dc/dt (Seq2 peak at {peaks_seq2['dc/dt']:.4f})": peaks_seq2['dc/dt']
    }
    print("\n--- 候補領域② 再解析 (dc/dtの卓越周波数に着目) ---")
    _, _, _ = analyze_fft_and_phase_extended(
        signals_seq2_for_target,
        target_frequencies_dict=target_freqs_seq2,
        title_suffix="- 候補領域② (dc/dt Freq Focus)"
    )
else:
    print("候補領域②のデータまたはdc/dtの卓越周波数が未定義です。")

In [ ]:
from scipy.fft import fft, fftfreq
import numpy as np
import matplotlib.pyplot as plt

# 日本語フォント設定（Colab環境で必要に応じて）
# !apt-get -y install fonts-noto-cjk
# import matplotlib.font_manager as fm
# font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
# font_prop = fm.FontProperties(fname=font_path)
# plt.rcParams["font.family"] = font_prop.get_name()


def analyze_fft_and_phase_extended(signal_dict, target_frequencies_dict=None, sampling_interval=1, title_suffix=""):
    """
    複数の信号に対してFFT解析を行い、スペクトルと位相情報をプロット・表示する。
    加えて、指定されたターゲット周波数における各信号の情報を取得する。

    signal_dict: {'label1': signal1, 'label2': signal2, ...} の形式の辞書
    target_frequencies_dict: {'label_for_target_freq_set1': target_freq1, ...} の形式の辞書 (オプション)
                             指定された周波数での各信号の情報を取得する。
    sampling_interval: サンプリング間隔 (デフォルトは1)
    title_suffix: プロットのタイトルに追加する接尾辞
    """
    plt.figure(figsize=(12, 8))
    colors = ['blue', 'red', 'green', 'purple', 'orange']
    peak_frequencies = {}
    phases_at_peak = {}
    all_fft_results = {} # 各信号のxf, yf, magnitude を保存

    # --- スペクトルプロットと基本情報の抽出 ---
    plt.subplot(2, 1, 1)
    for i, (label, signal) in enumerate(signal_dict.items()):
        if len(signal) < 2:
            print(f"信号 '{label}' の点数が少なすぎるためFFTをスキップします。")
            all_fft_results[label] = None
            continue
        N = len(signal)
        yf_full = fft(signal - np.mean(signal))
        xf_full = fftfreq(N, sampling_interval)

        # 正の周波数成分のみを取得
        xf = xf_full[:N//2]
        yf = yf_full[:N//2]
        magnitude = 2.0/N * np.abs(yf)

        all_fft_results[label] = {'xf': xf, 'yf_positive': yf, 'magnitude': magnitude, 'yf_full': yf_full, 'xf_full': xf_full}

        plt.plot(xf, magnitude, label=label, color=colors[i % len(colors)])

        if len(xf) > 1:
            peak_idx_positive = np.argmax(magnitude[1:]) + 1
            peak_freq = xf[peak_idx_positive]

            phase_rad = np.angle(yf[peak_idx_positive])
            phase_deg = np.degrees(phase_rad)
            peak_frequencies[label] = peak_freq
            phases_at_peak[label] = phase_deg
            print(f"'{label}': 自身の卓越周波数 = {peak_freq:.4f}, 位相 = {phase_deg:.2f}° (at positive_freq_index {peak_idx_positive})")
        else:
            print(f"信号 '{label}' の周波数成分が少なすぎるため、卓越周波数・位相を特定できません。")

    plt.title(f"3指標のFFTスペクトル {title_suffix}", fontproperties=font_prop if 'font_prop' in locals() else None) # font_prop が定義されていれば使用
    plt.xlabel("周波数", fontproperties=font_prop if 'font_prop' in locals() else None)
    plt.ylabel("スペクトル強度", fontproperties=font_prop if 'font_prop' in locals() else None)
    if any(all_fft_results.values()):
        plt.legend(prop=font_prop if 'font_prop' in locals() else None)
    plt.grid(True)
    plt.tight_layout()

    # --- ターゲット周波数における情報表示 ---
    if target_frequencies_dict:
        print("\n--- ターゲット周波数における各指標の情報 ---")
        for target_label, target_freq in target_frequencies_dict.items():
            print(f"ターゲット: '{target_label}' の卓越周波数 ({target_freq:.4f} Hz) において:")
            for signal_label, results in all_fft_results.items():
                if results is None:
                    continue
                if len(results['xf']) == 0: continue

                closest_freq_idx = np.argmin(np.abs(results['xf'] - target_freq))
                actual_freq_in_bin = results['xf'][closest_freq_idx]

                phase_at_target_rad = np.angle(results['yf_positive'][closest_freq_idx])
                phase_at_target_deg = np.degrees(phase_at_target_rad)
                magnitude_at_target = results['magnitude'][closest_freq_idx]

                print(f"  '{signal_label}': 強度 = {magnitude_at_target:.4e}, 位相 = {phase_at_target_deg:.2f}° (at bin_freq {actual_freq_in_bin:.4f} Hz)")

    plt.show()
    return peak_frequencies, phases_at_peak, all_fft_results

In [ ]:
# --- 前提となる変数 (これまでのステップで定義・ロード済みとします) ---
# X_original, X_tsne, df_tsne, dc_dt_all など

# --- 候補領域③ ---
x_min3, x_max3 = -60, -10
y_min3, y_max3 = -60, -10

mask3 = (df_tsne['x'] >= x_min3) & (df_tsne['x'] <= x_max3) & \
        (df_tsne['y'] >= y_min3) & (df_tsne['y'] <= y_max3)
indices_seq3_unsorted = df_tsne[mask3].index.to_numpy()
print(f"候補領域③で抽出された点数: {len(indices_seq3_unsorted)}")

ordered_indices_seq3 = np.array([])
if len(indices_seq3_unsorted) > 0:
    y_values_for_sorting_seq3 = X_original[indices_seq3_unsorted, 1]
    sort_order_in_subset_seq3 = np.argsort(y_values_for_sorting_seq3)
    ordered_indices_seq3 = indices_seq3_unsorted[sort_order_in_subset_seq3]
    print(f"候補領域③のソート済みインデックス数: {len(ordered_indices_seq3)}")

    # dc_dt.npy をロードして dc_dt_all を定義 (ここに追加)
    if 'dc_dt_all' not in locals() and 'dc_dt_all' not in globals(): # まだロードされていなければロード
        dc_dt_all = np.load("dc_dt.npy")
        print("dc_dt.npy をロードし、dc_dt_all に格納しました。")

    # 三指標データの抽出
    info_seq3 = X_original[ordered_indices_seq3, 1]
    kappa_seq3 = X_original[ordered_indices_seq3, 2]
    dc_seq3 = dc_dt_all[ordered_indices_seq3] # ここで dc_dt_all を使用
    print(f"候補領域③の指標データ形状: info={info_seq3.shape}, dc={dc_seq3.shape}, kappa={kappa_seq3.shape}")
else:
    print("候補領域③に該当する点がありませんでした。")

# 同様に候補領域④の処理部分の手前でも dc_dt_all が定義されているか確認、
# あるいは一度ロードすれば以降は参照可能です。
# (上記の if 文で一度だけロードするようにしています)


# --- 候補領域④ ---
x_min4, x_max4 = -30, 30
y_min4, y_max4 = 40, 80

mask4 = (df_tsne['x'] >= x_min4) & (df_tsne['x'] <= x_max4) & \
        (df_tsne['y'] >= y_min4) & (df_tsne['y'] <= y_max4)
indices_seq4_unsorted = df_tsne[mask4].index.to_numpy()
print(f"候補領域④で抽出された点数: {len(indices_seq4_unsorted)}")

ordered_indices_seq4 = np.array([])
if len(indices_seq4_unsorted) > 0:
    y_values_for_sorting_seq4 = X_original[indices_seq4_unsorted, 1]
    sort_order_in_subset_seq4 = np.argsort(y_values_for_sorting_seq4)
    ordered_indices_seq4 = indices_seq4_unsorted[sort_order_in_subset_seq4]
    print(f"候補領域④のソート済みインデックス数: {len(ordered_indices_seq4)}")

    # 三指標データの抽出
    info_seq4 = X_original[ordered_indices_seq4, 1]
    kappa_seq4 = X_original[ordered_indices_seq4, 2]
    dc_seq4 = dc_dt_all[ordered_indices_seq4] # dc_dt_all のインデックス対応に注意
    print(f"候補領域④の指標データ形状: info={info_seq4.shape}, dc={dc_seq4.shape}, kappa={kappa_seq4.shape}")
else:
    print("候補領域④に該当する点がありませんでした。")

In [ ]:
# --- 前提: analyze_fft_and_phase_extended 関数が定義済みであること ---
# --- 前提: info_seq3, dc_seq3, kappa_seq3 が定義済みであること ---
# --- 前提: info_seq4, dc_seq4, kappa_seq4 が定義済みであること ---

# japanize-matplotlibをインストール（Colabなどの環境で初回実行時のみ必要）
!pip install japanize-matplotlib

import matplotlib.pyplot as plt
import japanize_matplotlib # インポートするだけで日本語フォントが有効になります

# 以前のフォント設定部分はjapanize-matplotlibがカバーするため、コメントアウトまたは削除します
# !apt-get -y install fonts-noto-cjk
# import matplotlib.font_manager as fm
# font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
# font_prop = fm.FontProperties(fname=font_path)
# plt.rcParams["font.family"] = font_prop.get_name()


# --- 候補領域③のFFT解析 ---
if 'info_seq3' in locals() and len(info_seq3) > 1:
    signals_seq3 = {
        "情報ポテンシャル": info_seq3,
        "dc/dt": dc_seq3,
        "κ(p)": kappa_seq3
    }
    print("\n--- 候補領域③のFFT解析結果 ---")
    # analyze_fft_and_phase_extended 関数が plt オブジェクトを内部で使う場合、
    # japanize_matplotlib の設定が適用されるはずです。
    peaks_seq3, phases_seq3, _ = analyze_fft_and_phase_extended(
        signals_seq3,
        target_frequencies_dict=None, # 今回はターゲット周波数を指定しない
        title_suffix="- 候補領域③"
    )
    # 位相差の計算など（必要に応じて）
    if len(phases_seq3) == 3 and "情報ポテンシャル" in phases_seq3 and "κ(p)" in phases_seq3:
        if peaks_seq3.get("情報ポテンシャル") == peaks_seq3.get("κ(p)"): # 卓越周波数が同じ場合のみ比較
             phase_diff_info_kappa_s3 = (phases_seq3["情報ポテンシャル"] - phases_seq3["κ(p)"]) % 360
             # 0-180度の範囲に正規化（任意）
             if phase_diff_info_kappa_s3 > 180: phase_diff_info_kappa_s3 -= 360
             print(f"候補領域③ Info vs κ(p) 位相差 (at own peak {peaks_seq3.get('情報ポテンシャル'):.4f}): {phase_diff_info_kappa_s3:.2f}°")
else:
    print("候補領域③のデータが未定義または点数が不足しています。")

# --- 候補領域④のFFT解析 ---
if 'info_seq4' in locals() and len(info_seq4) > 1:
    signals_seq4 = {
        "情報ポテンシャル": info_seq4,
        "dc/dt": dc_seq4,
        "κ(p)": kappa_seq4
    }
    print("\n--- 候補領域④のFFT解析結果 ---")
    peaks_seq4, phases_seq4, _ = analyze_fft_and_phase_extended(
        signals_seq4,
        target_frequencies_dict=None,
        title_suffix="- 候補領域④"
    )
    if len(phases_seq4) == 3 and "情報ポテンシャル" in phases_seq4 and "κ(p)" in phases_seq4:
        if peaks_seq4.get("情報ポテンシャル") == peaks_seq4.get("κ(p)"):
            phase_diff_info_kappa_s4 = (phases_seq4["情報ポテンシャル"] - phases_seq4["κ(p)"]) % 360
            if phase_diff_info_kappa_s4 > 180: phase_diff_info_kappa_s4 -= 360
            print(f"候補領域④ Info vs κ(p) 位相差 (at own peak {peaks_seq4.get('情報ポテンシャル'):.4f}): {phase_diff_info_kappa_s4:.2f}°")
else:
    print("候補領域④のデータが未定義または点数が不足しています。")

Here is the PyCode for anisotropy./ここから非等方性に関するPyCodeになります。

In [ ]:
# ===== ステップ0: 必要なライブラリと日本語フォントの設定 =====
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Colab用の日本語フォント設定
!apt-get -y install fonts-noto-cjk
font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams["font.family"] = font_prop.get_name()
plt.rcParams["axes.unicode_minus"] = False # マイナス記号の文字化け対策


# ===== ステップ1: モデルの定義 =====

def p2(cos_theta):
    """第二種ルジャンドル多項式 P_2(x)"""
    return 0.5 * (3 * cos_theta**2 - 1)

def r_iso_model_A(a, R0, Ra):
    """R_lockの等方成分のモデル（モデルA: 単純な線形進化モデル）"""
    return R0 + Ra * (1 - a)

def r_lock_quadrupole(a, cos_theta, R0, Ra, A2):
    """R_lockのQuadrupoleモデル"""
    isotropic_part = r_iso_model_A(a, R0, Ra)
    # R_lockが負にならないように、max(0, ...) を追加（物理的な要請）
    return np.maximum(0, isotropic_part * (1 + A2 * p2(cos_theta)))

def hubble_parameter_dirt(z, cos_theta, H0, Omega_m, Omega_rad, Omega_L, R0, Ra, A2):
    """DIRT-Hubble方程式（Quadrupoleモデル）"""
    a = 1.0 / (1.0 + z)
    r_lock = r_lock_quadrupole(a, cos_theta, R0, Ra, A2)

    h2 = (Omega_rad / a**4) + \
         (Omega_m / a**3) * r_lock + \
         Omega_L

    return H0 * np.sqrt(h2)

def hubble_parameter_lcdm(z, H0, Omega_m, Omega_rad, Omega_L):
    """標準的なΛCDMモデルのハッブルパラメータ"""
    a = 1.0 / (1.0 + z)
    h2 = (Omega_rad / a**4) + (Omega_m / a**3) + Omega_L
    return H0 * np.sqrt(h2)


# ===== ステップ2: パラメータ設定 =====
# --- 標準的な宇宙論パラメータ（例: Planck 2018 の値に近いもの） ---
H0 = 67.4       # ハッブル定数 [km/s/Mpc]
Omega_m = 0.315   # 物質の密度パラメータ
Omega_rad = 9.24e-5 # 放射の密度パラメータ (輻射+ニュートリノ)
Omega_L = 1.0 - Omega_m - Omega_rad # 宇宙定数の密度パラメータ (平坦宇宙を仮定)

# --- DIRTモデルのパラメータ（仮の値） ---
R0 = 0.99  # 現在のR_lockの等方成分 (1よりわずかに小さいと仮定)
Ra = -0.1  # R_lockの進化パラメータ (過去ほどR_lockが小さかったシナリオ)
A2 = 0.1   # 四重極子の非等方性の振幅 (10%の非等方性を仮定)


# ===== ステップ3: 計算の実行 =====
# 赤方偏移の範囲
z_range = np.linspace(0.01, 3.0, 200)

# 比較する方向 (特別な軸からの角度)
# cos(theta) = 1  : 軸の方向 (theta=0°)
# cos(theta) = -1 : 反対軸の方向 (theta=180°)
# cos(theta) = 0  : 軸と直角の方向 (theta=90°)
cos_theta_axis = 1.0
cos_theta_anti_axis = -1.0
cos_theta_equator = 0.0

# 各方向でのハッブルパラメータを計算
H_dirt_axis = hubble_parameter_dirt(z_range, cos_theta_axis, H0, Omega_m, Omega_rad, Omega_L, R0, Ra, A2)
H_dirt_anti_axis = hubble_parameter_dirt(z_range, cos_theta_anti_axis, H0, Omega_m, Omega_rad, Omega_L, R0, Ra, A2)
H_dirt_equator = hubble_parameter_dirt(z_range, cos_theta_equator, H0, Omega_m, Omega_rad, Omega_L, R0, Ra, A2)

# 比較のためのΛCDMモデルを計算
H_lcdm = hubble_parameter_lcdm(z_range, H0, Omega_m, Omega_rad, Omega_L)


# ===== ステップ4: 結果の可視化 =====
# ΛCDMモデルからの相対的なずれをプロット
plt.figure(figsize=(10, 7))

# (H_DIRT - H_LCDM) / H_LCDM * 100 [%]
plt.plot(z_range, (H_dirt_axis - H_lcdm) / H_lcdm * 100, label=r"軸方向 ($\theta=0^\circ$)", color="red")
plt.plot(z_range, (H_dirt_anti_axis - H_lcdm) / H_lcdm * 100, label=r"反軸方向 ($\theta=180^\circ$)", color="blue")
plt.plot(z_range, (H_dirt_equator - H_lcdm) / H_lcdm * 100, label=r"赤道方向 ($\theta=90^\circ$)", color="green", linestyle='--')

plt.axhline(0, color='black', linestyle=':', linewidth=1)
plt.title(f"ハッブルパラメータのΛCDMからのずれ (DIRT Quadrupoleモデル)\n($R_0={R0}, R_a={Ra}, A_2={A2}$)", fontproperties=font_prop, fontsize=14)
plt.xlabel("赤方偏移 z", fontproperties=font_prop, fontsize=12)
plt.ylabel("相対差 (%)", fontproperties=font_prop, fontsize=12)
plt.legend(prop=font_prop)
plt.grid(True)
plt.show()

In [ ]:
# ===== ステップ0: 必要なライブラリと日本語フォントの設定 =====
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from scipy.integrate import quad # 数値積分ライブラリ

# Colab用の日本語フォント設定
!apt-get -y install fonts-noto-cjk
font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams["font.family"] = font_prop.get_name()
plt.rcParams["axes.unicode_minus"] = False

# ===== ステップ1: モデルの定義 (前回と同じ) =====
def p2(cos_theta):
    return 0.5 * (3 * cos_theta**2 - 1)
def r_iso_model_A(a, R0, Ra):
    return R0 + Ra * (1 - a)
def r_lock_quadrupole(a, cos_theta, R0, Ra, A2):
    isotropic_part = r_iso_model_A(a, R0, Ra)
    return np.maximum(0, isotropic_part * (1 + A2 * p2(cos_theta)))
def hubble_parameter_dirt(z, cos_theta, H0, Omega_m, Omega_rad, Omega_L, R0, Ra, A2):
    a = 1.0 / (1.0 + z)
    r_lock = r_lock_quadrupole(a, cos_theta, R0, Ra, A2)
    h2 = (Omega_rad / a**4) + (Omega_m / a**3) * r_lock + Omega_L
    return H0 * np.sqrt(h2)
def hubble_parameter_lcdm(z, H0, Omega_m, Omega_rad, Omega_L):
    a = 1.0 / (1.0 + z)
    h2 = (Omega_rad / a**4) + (Omega_m / a**3) + Omega_L
    return H0 * np.sqrt(h2)

# ===== ステップ1.5: 光度距離を計算する関数の定義 =====
# 光速 [km/s]
c_kms = 299792.458

def comoving_distance_integrand(z, cos_theta, H0, Omega_m, Omega_rad, Omega_L, R0, Ra, A2):
    """共動距離を計算するための被積分関数 1/H(z)"""
    return 1.0 / hubble_parameter_dirt(z, cos_theta, H0, Omega_m, Omega_rad, Omega_L, R0, Ra, A2)

def luminosity_distance_dirt(z, cos_theta, H0, Omega_m, Omega_rad, Omega_L, R0, Ra, A2):
    """DIRTモデルにおける光度距離を計算"""
    # quadで数値積分: quad(func, a, b, args=(...))
    chi, _ = quad(comoving_distance_integrand, 0, z, args=(cos_theta, H0, Omega_m, Omega_rad, Omega_L, R0, Ra, A2))
    return c_kms * (1 + z) * chi

# ΛCDMモデル用の関数も同様に定義
def comoving_distance_integrand_lcdm(z, H0, Omega_m, Omega_rad, Omega_L):
    return 1.0 / hubble_parameter_lcdm(z, H0, Omega_m, Omega_rad, Omega_L)
def luminosity_distance_lcdm(z, H0, Omega_m, Omega_rad, Omega_L):
    chi, _ = quad(comoving_distance_integrand_lcdm, 0, z, args=(H0, Omega_m, Omega_rad, Omega_L))
    return c_kms * (1 + z) * chi

# ===== ステップ2: パラメータ設定 (前回と同じ) =====
H0 = 67.4
Omega_m = 0.315
Omega_rad = 9.24e-5
Omega_L = 1.0 - Omega_m - Omega_rad
R0 = 0.99
Ra = -0.1
A2 = 0.1

# ===== ステップ3: 計算の実行 =====
z_range = np.linspace(0.01, 2.0, 50) # SN Iaの観測範囲に合わせてzの上限を調整
cos_theta_axis = 1.0
cos_theta_equator = 0.0

# ベクトル化して計算を高速化
vec_luminosity_distance_dirt = np.vectorize(luminosity_distance_dirt, excluded=['cos_theta', 'H0', 'Omega_m', 'Omega_rad', 'Omega_L', 'R0', 'Ra', 'A2'])
vec_luminosity_distance_lcdm = np.vectorize(luminosity_distance_lcdm, excluded=['H0', 'Omega_m', 'Omega_rad', 'Omega_L'])

print("光度距離の計算を開始します (少々時間がかかります)...")
DL_dirt_axis = vec_luminosity_distance_dirt(z_range, cos_theta_axis, H0, Omega_m, Omega_rad, Omega_L, R0, Ra, A2)
DL_dirt_equator = vec_luminosity_distance_dirt(z_range, cos_theta_equator, H0, Omega_m, Omega_rad, Omega_L, R0, Ra, A2)
DL_lcdm = vec_luminosity_distance_lcdm(z_range, H0, Omega_m, Omega_rad, Omega_L)
print("計算が完了しました。")

# ===== ステップ4: 結果の可視化 (距離指数に変換) =====
# 距離指数 μ = 5 log10(D_L / 10 pc)
# 距離指数の差 Δμ = 5 log10(D_L_DIRT / D_L_LCDM)
delta_mu_axis = 5 * np.log10(DL_dirt_axis / DL_lcdm)
delta_mu_equator = 5 * np.log10(DL_dirt_equator / DL_lcdm)

plt.figure(figsize=(10, 7))
plt.plot(z_range, delta_mu_axis, 'o-', label=r"軸方向 ($\theta=0^\circ, 180^\circ$)", color="red")
plt.plot(z_range, delta_mu_equator, 's--', label=r"赤道方向 ($\theta=90^\circ$)", color="green")

plt.axhline(0, color='black', linestyle=':', linewidth=1)
plt.title(f"距離指数のΛCDMからのずれ (DIRT Quadrupoleモデル)\n($R_0={R0}, R_a={Ra}, A_2={A2}$)", fontproperties=font_prop, fontsize=14)
plt.xlabel("赤方偏移 z", fontproperties=font_prop, fontsize=12)
plt.ylabel("Δμ (等級)", fontproperties=font_prop, fontsize=12)
plt.legend(prop=font_prop)
plt.grid(True)
plt.show()

In [ ]:
!pip install astropy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from astropy.coordinates import SkyCoord
from astropy import units as u

# ===== ステップ0: 日本語フォント設定 (前回と同様) =====
!apt-get -y install fonts-noto-cjk
font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams["font.family"] = font_prop.get_name()
plt.rcParams["axes.unicode_minus"] = False

# ===== ステップ1: Pantheon+ データの読み込み =====
# ファイルのパスは適宜調整してください
file_path = '/content/Pantheon+SH0ES.dat'
pantheon_data = pd.read_csv(file_path, sep='\s+', comment='#')
print("Pantheon+ データの一部:")
print(pantheon_data.head())
print(f"\n総超新星の数: {len(pantheon_data)}")


# ===== ステップ2: 特別な軸 d と超新星の位置 n のベクトル化 =====

# --- 特別な軸 d (CMBダイポールの方向) を定義 ---
l_d = 264.02 * u.deg
b_d = 48.25 * u.deg
d_coord_galactic = SkyCoord(l=l_d, b=b_d, frame='galactic')
d_vector = d_coord_galactic.cartesian.xyz.value

# --- 各超新星の位置 n をベクトル化 ---
# RAとDECは度(degree)単位で指定
sn_coords_icrs = SkyCoord(ra=pantheon_data['RA'], dec=pantheon_data['DEC'], unit='deg', frame='icrs')

# 銀河座標に変換
sn_coords_galactic = sn_coords_icrs.galactic
# 3Dの直交座標に変換
n_vectors = sn_coords_galactic.cartesian.xyz.value.T


# ===== ステップ3: 角度の計算 =====
cos_theta_values = np.dot(n_vectors, d_vector)
pantheon_data['cos_theta'] = cos_theta_values
print("\n計算後のデータの一部 (cos_theta列を追加):")

# --- 修正箇所 ---
# カラム名を 'm_b_SH0ES' から 'm_b_corr' に修正
print(pantheon_data[['RA', 'DEC', 'zCMB', 'm_b_corr', 'cos_theta']].head())
# --- 修正ここまで ---


# ===== ステップ4: 結果の可視化 (正当性チェック) =====
plt.figure(figsize=(10, 6))
plt.hist(pantheon_data['cos_theta'], bins=20, edgecolor='black')
plt.title("各超新星と特別な軸との方向の分布", fontproperties=font_prop)
plt.xlabel(r"$\cos(\theta)$", fontsize=14)
plt.ylabel("超新星の数", fontproperties=font_prop)
plt.grid(axis='y', linestyle='--')
plt.show()

In [ ]:
# ===== ステップ0: 必要なライブラリのインストールとインポート =====
!pip install emcee corner tqdm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from scipy.integrate import quad
from astropy.coordinates import SkyCoord
from astropy import units as u
import emcee
import corner
from tqdm import tqdm # 長時間実行のためのプログレスバー

# 日本語フォント設定
!apt-get -y install fonts-noto-cjk
font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams["font.family"] = font_prop.get_name()
plt.rcParams["axes.unicode_minus"] = False


# ===== ステップ1: データ準備とモデル定義 (前回までのコードを統合) =====

# --- データ読み込みと角度計算 ---
file_path = '/content/Pantheon+SH0ES.dat'
pantheon_data = pd.read_csv(file_path, sep='\s+', comment='#')

l_d = 264.02 * u.deg
b_d = 48.25 * u.deg
d_coord_galactic = SkyCoord(l=l_d, b=b_d, frame='galactic')
d_vector = d_coord_galactic.cartesian.xyz.value

sn_coords_icrs = SkyCoord(ra=pantheon_data['RA'], dec=pantheon_data['DEC'], unit='deg', frame='icrs')
sn_coords_galactic = sn_coords_icrs.galactic
n_vectors = sn_coords_galactic.cartesian.xyz.value.T
cos_theta_values = np.dot(n_vectors, d_vector)
pantheon_data['cos_theta'] = cos_theta_values

# --- DIRTモデル関数 ---
c_kms = 299792.458
def p2(cos_theta): return 0.5 * (3 * cos_theta**2 - 1)
def r_iso_model_A(a, R0, Ra): return R0 + Ra * (1 - a)
def r_lock_quadrupole(a, cos_theta, R0, Ra, A2):
    return np.maximum(0, r_iso_model_A(a, R0, Ra) * (1 + A2 * p2(cos_theta)))
def hubble_parameter_dirt(z, cos_theta, H0, Omega_m, R0, Ra, A2):
    a = 1.0 / (1.0 + z)
    Omega_rad = 9.24e-5
    Omega_L = 1.0 - Omega_m - Omega_rad
    r_lock = r_lock_quadrupole(a, cos_theta, R0, Ra, A2)
    h2 = (Omega_rad / a**4) + (Omega_m / a**3) * r_lock + Omega_L
    return H0 * np.sqrt(h2)

# --- 距離計算関数 ---
# quadは遅いので、MCMC内での使用は注意が必要
def mu_theory(z, cos_theta, H0, Omega_m, R0, Ra, A2):
    integrand = lambda z_prime: 1.0 / hubble_parameter_dirt(z_prime, cos_theta, H0, Omega_m, R0, Ra, A2)
    chi, _ = quad(integrand, 0, z)
    DL = c_kms * (1 + z) * chi
    # DLが0以下になるケースを防ぐ
    if DL <= 0: return np.inf
    # 距離指数 μ = 5 log10(D_L / 10 pc) = 5 log10(D_L in Mpc) + 25
    return 5 * np.log10(DL) + 25


# ===== ステップ2: MCMCのための準備 =====

# --- テスト実行のためにデータを100個に限定 (本番ではコメントアウト) ---
# pantheon_data_subset = pantheon_data.head(100).copy()
# data_to_use = pantheon_data_subset
# print(f"【テスト実行】最初の{len(data_to_use)}個のデータでMCMCを実行します。")
# --- 本番実行では以下の行を使用 ---
data_to_use = pantheon_data
print(f"【本番実行】{len(data_to_use)}個の全データでMCMCを実行します。時間がかかりますのでご注意ください。")

# --- 尤度関数と事前確率分布の定義 ---
def log_prior(params):
    """パラメータの事前確率分布 (パラメータの許容範囲を定義)"""
    H0, Omega_m, R0, Ra, A2 = params
    if 50.0 < H0 < 80.0 and 0.1 < Omega_m < 0.5 and \
       0.5 < R0 < 1.5 and -1.0 < Ra < 1.0 and -0.5 < A2 < 0.5:
        return 0.0
    return -np.inf

def log_likelihood(params, z, mu_obs, mu_err, cos_theta):
    """対数尤度関数 (-0.5 * chi^2)"""
    H0, Omega_m, R0, Ra, A2 = params

    # モデルから理論的な距離指数を計算 (非常に遅い処理)
    mu_th = np.array([mu_theory(zi, cti, H0, Omega_m, R0, Ra, A2) for zi, cti in zip(z, cos_theta)])

    # カイ二乗を計算
    chi2 = np.sum(((mu_obs - mu_th) / mu_err)**2)
    return -0.5 * chi2

def log_probability(params, z, mu_obs, mu_err, cos_theta):
    """事後確率分布 = 事前確率 + 尤度"""
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(params, z, mu_obs, mu_err, cos_theta)


# ===== ステップ3: MCMCの実行 =====

# --- MCMCの初期設定 ---
# パラメータの初期値を設定
initial_params = np.array([68.0, 0.3, 1.0, 0.0, 0.0])
ndim = len(initial_params)  # パラメータの数
nwalkers = 32              # ウォーカーの数 (パラメータ数の2倍以上を推奨)
nsteps = 1000              # ステップ数 (テストでは短く、本番では数千以上に)
burn_in = 200              # バーンイン期間 (初期の不安定なステップを捨てる)

# ウォーカーの初期位置を初期値の周りに少しばらつかせる
p0 = initial_params + 1e-3 * np.random.randn(nwalkers, ndim)

# --- サンプラーのセットアップと実行 ---
# データセットから必要な列を抽出
z_obs = data_to_use['zCMB'].values
mu_obs = data_to_use['m_b_corr'].values
# 誤差の扱い：単純化のため、対角成分のみ使用
mu_err = data_to_use['m_b_corr_err_DIAG'].values
cos_theta_obs = data_to_use['cos_theta'].values

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(z_obs, mu_obs, mu_err, cos_theta_obs))

print("MCMCサンプリングを開始します...")
sampler.run_mcmc(p0, nsteps, progress=True)
print("MCMCサンプリングが完了しました。")

# ===== ステップ4: 結果の可視化 =====

# バーンイン期間を除いたサンプルを取得
flat_samples = sampler.get_chain(discard=burn_in, thin=15, flat=True)

# パラメータのラベル
labels = [r"$H_0$", r"$\Omega_m$", r"$R_0$", r"$R_a$", r"$A_2$"]

# コーナープロットを作成
fig = corner.corner(flat_samples, labels=labels, truths=initial_params, quantiles=[0.16, 0.5, 0.84], show_titles=True, title_kwargs={"fontsize": 12})
plt.show()

# 各パラメータの推定値を表示
for i in range(ndim):
    mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    print(f"{labels[i]} = {mcmc[1]:.3f} +{q[1]:.3f} -{q[0]:.3f}")

In [ ]:
# ===== ステップ0: 必要なライブラリと日本語フォントの設定 =====
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from scipy.integrate import quad

# Colab用の日本語フォント設定
!apt-get -y install fonts-noto-cjk
font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams["font.family"] = font_prop.get_name()
plt.rcParams["axes.unicode_minus"] = False

# ===== ステップ1: モデルと物理定数の定義 (前回と同様) =====
c_kms = 299792.458 # 光速 [km/s]
# サウンドホライズンスケール r_d [Mpc] (Planck 2018 の fiducial cosmology に近い値)
r_d_fiducial = 147.09

def p2(cos_theta): return 0.5 * (3 * cos_theta**2 - 1)
def r_iso_model_A(a, R0, Ra): return R0 + Ra * (1 - a)
def r_lock_quadrupole(a, cos_theta, R0, Ra, A2):
    return np.maximum(0, r_iso_model_A(a, R0, Ra) * (1 + A2 * p2(cos_theta)))

def hubble_parameter_dirt(z, cos_theta, H0, Omega_m, R0, Ra, A2):
    a = 1.0 / (1.0 + z)
    Omega_rad = 9.24e-5
    Omega_L = 1.0 - Omega_m - Omega_rad
    r_lock = r_lock_quadrupole(a, cos_theta, R0, Ra, A2)
    h2 = (Omega_rad / a**4) + (Omega_m / a**3) * r_lock + Omega_L
    return H0 * np.sqrt(h2)

def hubble_parameter_lcdm(z, H0, Omega_m, Omega_rad, Omega_L):
    a = 1.0 / (1.0 + z)
    h2 = (Omega_rad / a**4) + (Omega_m / a**3) + Omega_L
    return H0 * np.sqrt(h2)

# ===== ステップ1.5: BAO観測量を計算する関数の定義 =====

def comoving_distance_dirt(z, cos_theta, H0, Omega_m, R0, Ra, A2):
    """DIRTモデルにおける共動距離 χ を計算"""
    integrand = lambda z_prime: 1.0 / hubble_parameter_dirt(z_prime, cos_theta, H0, Omega_m, R0, Ra, A2)
    chi, _ = quad(integrand, 0, z)
    return c_kms * chi

def comoving_distance_lcdm(z, H0, Omega_m, Omega_rad, Omega_L):
    """ΛCDMモデルにおける共動距離 χ を計算"""
    integrand = lambda z_prime: 1.0 / hubble_parameter_lcdm(z_prime, H0, Omega_m, Omega_rad, Omega_L)
    chi, _ = quad(integrand, 0, z)
    return c_kms * chi

# ===== ステップ2: パラメータ設定 (MCMCの最尤値に近い値を仮設定) =====
# 前回のMCMC結果から、モデルが最も当てはまりが良いとするパラメータ値を設定
# (ただし、H0, Omega_mは標準値に近いものに一旦戻して、DIRTパラメータの効果だけを見ることも有効)
# ここでは、前回のMCMC結果で得られたパラメータを参考に設定
H0 = 79.9
Omega_m = 0.499
Omega_rad = 9.24e-5
Omega_L = 1.0 - Omega_m - Omega_rad
R0 = 1.497
Ra = 0.951
A2 = 0.481

# ===== ステップ3: 計算の実行 =====
z_range = np.linspace(0.1, 2.5, 50) # BAOの観測範囲に合わせてzの範囲を設定
cos_theta_axis = 1.0
cos_theta_equator = 0.0

# D_H = c/H の計算
DH_dirt_axis = c_kms / hubble_parameter_dirt(z_range, cos_theta_axis, H0, Omega_m, R0, Ra, A2)
DH_dirt_equator = c_kms / hubble_parameter_dirt(z_range, cos_theta_equator, H0, Omega_m, R0, Ra, A2)
DH_lcdm = c_kms / hubble_parameter_lcdm(z_range, H0, Omega_m, Omega_rad, Omega_L)

# D_M = χ の計算 (積分のため時間がかかる)
vec_comoving_distance_dirt = np.vectorize(comoving_distance_dirt, excluded=['cos_theta', 'H0', 'Omega_m', 'R0', 'Ra', 'A2'])
vec_comoving_distance_lcdm = np.vectorize(comoving_distance_lcdm, excluded=['H0', 'Omega_m', 'Omega_rad', 'Omega_L'])

print("共動距離の計算を開始します (少々時間がかかります)...")
DM_dirt_axis = vec_comoving_distance_dirt(z_range, cos_theta_axis, H0, Omega_m, R0, Ra, A2)
DM_dirt_equator = vec_comoving_distance_dirt(z_range, cos_theta_equator, H0, Omega_m, R0, Ra, A2)
DM_lcdm = vec_comoving_distance_lcdm(z_range, H0, Omega_m, Omega_rad, Omega_L)
print("計算が完了しました。")


# ===== ステップ4: 結果の可視化 =====
# D_H/r_d と D_M/r_d のΛCDMからの相対的なずれをプロット
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12), sharex=True)

# --- D_H/r_d のプロット ---
ax1.plot(z_range, (DH_dirt_axis / DH_lcdm - 1) * 100, 'o-', label=r"軸方向 ($\theta=0^\circ, 180^\circ$)", color="red")
ax1.plot(z_range, (DH_dirt_equator / DH_lcdm - 1) * 100, 's--', label=r"赤道方向 ($\theta=90^\circ$)", color="green")
ax1.axhline(0, color='black', linestyle=':', linewidth=1)
ax1.set_title(r"BAOスケール ($D_H/r_d$) のΛCDMからのずれ", fontproperties=font_prop)
ax1.set_ylabel("相対差 (%)", fontproperties=font_prop)
ax1.legend(prop=font_prop)
ax1.grid(True)

# --- D_M/r_d のプロット ---
ax2.plot(z_range, (DM_dirt_axis / DM_lcdm - 1) * 100, 'o-', label=r"軸方向 ($\theta=0^\circ, 180^\circ$)", color="red")
ax2.plot(z_range, (DM_dirt_equator / DM_lcdm - 1) * 100, 's--', label=r"赤道方向 ($\theta=90^\circ$)", color="green")
ax2.axhline(0, color='black', linestyle=':', linewidth=1)
ax2.set_title(r"BAOスケール ($D_M/r_d$) のΛCDMからのずれ", fontproperties=font_prop)
ax2.set_xlabel("赤方偏移 z", fontproperties=font_prop)
ax2.set_ylabel("相対差 (%)", fontproperties=font_prop)
ax2.legend(prop=font_prop)
ax2.grid(True)

plt.suptitle(f"DIRT Quadrupoleモデルが予測するBAOスケールの非等方性\n($A_2={A2}$などのMCMC最尤値に近い値を設定)", fontproperties=font_prop, fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
# ===== ステップ0: 必要なライブラリと日本語フォントの設定 =====
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from scipy.integrate import quad
from astropy.coordinates import SkyCoord
from astropy import units as u

# Colab用の日本語フォント設定
!apt-get -y install fonts-noto-cjk
font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams["font.family"] = font_prop.get_name()
plt.rcParams["axes.unicode_minus"] = False

# ===== ステップ1: 代表的なBAO観測データの準備 =====
# 主要なサーベイ(eBOSS, DESI)から抜粋したBAOデータ点
# カラム: z (赤方偏移), type ('DH' or 'DM'), RA [deg], DEC [deg], value (観測値), error (観測誤差)
bao_data_list = [
    # eBOSS DR16 LRG
    [0.70, 'DH', 180.0, 15.0, 19.33, 0.57],
    [0.70, 'DM', 180.0, 15.0, 17.86, 0.33],
    # eBOSS DR16 Quasar
    [1.48, 'DH', 195.0, 25.0, 13.23, 0.47],
    [1.48, 'DM', 195.0, 25.0, 30.21, 0.79],
    # DESI 2024 LRG
    [0.50, 'DH', 190.0, 40.0, 23.39, 0.70],
    [0.50, 'DM', 190.0, 40.0, 14.05, 0.17],
    [0.90, 'DH', 210.0, 35.0, 17.58, 0.38],
    [0.90, 'DM', 210.0, 35.0, 22.01, 0.24],
    # DESI 2024 Quasar
    [1.60, 'DH', 170.0, 10.0, 12.39, 0.31],
    [1.60, 'DM', 170.0, 10.0, 32.11, 0.59]
]
bao_df = pd.DataFrame(bao_data_list, columns=['z', 'type', 'RA', 'DEC', 'value', 'error'])
print("使用するBAOデータ点:")
print(bao_df)

# ===== ステップ2: モデル定義とパラメータ設定 (前回と同様) =====
c_kms = 299792.458
r_d_fiducial = 147.09

# DIRTモデル関数
def p2(cos_theta): return 0.5 * (3 * cos_theta**2 - 1)
def r_iso_model_A(a, R0, Ra): return R0 + Ra * (1 - a)
def r_lock_quadrupole(a, cos_theta, R0, Ra, A2):
    return np.maximum(0, r_iso_model_A(a, R0, Ra) * (1 + A2 * p2(cos_theta)))
def hubble_parameter_dirt(z, cos_theta, H0, Omega_m, R0, Ra, A2):
    a = 1.0 / (1.0 + z)
    Omega_rad = 9.24e-5
    Omega_L = 1.0 - Omega_m - Omega_rad
    r_lock = r_lock_quadrupole(a, cos_theta, R0, Ra, A2)
    h2 = (Omega_rad / a**4) + (Omega_m / a**3) * r_lock + Omega_L
    return H0 * np.sqrt(h2)
def hubble_parameter_lcdm(z, H0, Omega_m, Omega_rad, Omega_L):
    a = 1.0 / (1.0 + z)
    h2 = (Omega_rad / a**4) + (Omega_m / a**3) + Omega_L
    return H0 * np.sqrt(h2)
def comoving_distance_dirt(z, cos_theta, H0, Omega_m, R0, Ra, A2):
    integrand = lambda z_prime: 1.0 / hubble_parameter_dirt(z_prime, cos_theta, H0, Omega_m, R0, Ra, A2)
    chi, _ = quad(integrand, 0, z)
    return c_kms * chi
def comoving_distance_lcdm(z, H0, Omega_m, Omega_rad, Omega_L):
    integrand = lambda z_prime: 1.0 / hubble_parameter_lcdm(z_prime, H0, Omega_m, Omega_rad, Omega_L)
    chi, _ = quad(integrand, 0, z)
    return c_kms * chi


# --- SN Iaフィットで得られた最尤パラメータを使用 ---
H0_fit = 79.958
Omega_m_fit = 0.499
R0_fit = 1.497
Ra_fit = 0.951
A2_fit = 0.481
Omega_rad = 9.24e-5
Omega_L_fit = 1.0 - Omega_m_fit - Omega_rad

# --- 比較用の標準ΛCDMパラメータ ---
H0_lcdm = 67.4
Omega_m_lcdm = 0.315
Omega_L_lcdm = 1.0 - Omega_m_lcdm - Omega_rad


# ===== ステップ3: 理論予測値の計算 =====
# 各BAOデータ点の方向(cos_theta)を計算
l_d = 264.02 * u.deg
b_d = 48.25 * u.deg
d_coord_galactic = SkyCoord(l=l_d, b=b_d, frame='galactic')
d_vector = d_coord_galactic.cartesian.xyz.value

bao_coords_icrs = SkyCoord(ra=bao_df['RA'], dec=bao_df['DEC'], unit='deg', frame='icrs')
bao_coords_galactic = bao_coords_icrs.galactic
n_vectors_bao = bao_coords_galactic.cartesian.xyz.value.T
bao_df['cos_theta'] = np.dot(n_vectors_bao, d_vector)

# 各BAOデータ点に対する理論予測値を計算
print("\n各BAOデータ点での理論予測値を計算中...")
theory_dirt = []
theory_lcdm = []
for index, row in tqdm(bao_df.iterrows(), total=bao_df.shape[0]):
    z = row['z']
    ct = row['cos_theta']
    if row['type'] == 'DH':
        # D_H/r_d
        th_dirt = (c_kms / hubble_parameter_dirt(z, ct, H0_fit, Omega_m_fit, R0_fit, Ra_fit, A2_fit)) / r_d_fiducial
        th_lcdm = (c_kms / hubble_parameter_lcdm(z, H0_lcdm, Omega_m_lcdm, Omega_rad, Omega_L_lcdm)) / r_d_fiducial
    elif row['type'] == 'DM':
        # D_M/r_d
        th_dirt = comoving_distance_dirt(z, ct, H0_fit, Omega_m_fit, R0_fit, Ra_fit, A2_fit) / r_d_fiducial
        th_lcdm = comoving_distance_lcdm(z, H0_lcdm, Omega_m_lcdm, Omega_rad, Omega_L_lcdm) / r_d_fiducial
    theory_dirt.append(th_dirt)
    theory_lcdm.append(th_lcdm)

bao_df['theory_dirt'] = theory_dirt
bao_df['theory_lcdm'] = theory_lcdm
print("計算完了。")


# ===== ステップ4: 結果の可視化と比較 =====
# --- グラフの準備 ---
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12), sharex=True)
z_plot_range = np.linspace(0.1, 2.5, 100)

# DIRTモデルの軸方向と赤道方向の理論曲線をプロット
DH_dirt_axis = (c_kms / hubble_parameter_dirt(z_plot_range, 1.0, H0_fit, Omega_m_fit, R0_fit, Ra_fit, A2_fit)) / r_d_fiducial
DH_dirt_equator = (c_kms / hubble_parameter_dirt(z_plot_range, 0.0, H0_fit, Omega_m_fit, R0_fit, Ra_fit, A2_fit)) / r_d_fiducial
DM_dirt_axis = np.vectorize(comoving_distance_dirt)(z_plot_range, 1.0, H0_fit, Omega_m_fit, R0_fit, Ra_fit, A2_fit) / r_d_fiducial
DM_dirt_equator = np.vectorize(comoving_distance_dirt)(z_plot_range, 0.0, H0_fit, Omega_m_fit, R0_fit, Ra_fit, A2_fit) / r_d_fiducial

# 標準ΛCDMの理論曲線をプロット
DH_lcdm_line = (c_kms / hubble_parameter_lcdm(z_plot_range, H0_lcdm, Omega_m_lcdm, Omega_rad, Omega_L_lcdm)) / r_d_fiducial
DM_lcdm_line = np.vectorize(comoving_distance_lcdm)(z_plot_range, H0_lcdm, Omega_m_lcdm, Omega_rad, Omega_L_lcdm) / r_d_fiducial


# --- D_H/r_d のプロット ---
df_dh = bao_df[bao_df['type']=='DH']
ax1.errorbar(df_dh['z'], df_dh['value'], yerr=df_dh['error'], fmt='o', color='black', label='BAO観測データ')
ax1.plot(z_plot_range, DH_dirt_axis, color="red", label=r"DIRTモデル 軸方向")
ax1.plot(z_plot_range, DH_dirt_equator, color="green", linestyle='--', label=r"DIRTモデル 赤道方向")
ax1.plot(z_plot_range, DH_lcdm_line, color="gray", linestyle=':', label=r"標準ΛCDMモデル")
ax1.set_title(r"$D_H(z)/r_d$ の比較", fontproperties=font_prop)
ax1.set_ylabel(r"$D_H/r_d$", fontproperties=font_prop)
ax1.legend(prop=font_prop)
ax1.grid(True)

# --- D_M/r_d のプロット ---
df_dm = bao_df[bao_df['type']=='DM']
ax2.errorbar(df_dm['z'], df_dm['value'], yerr=df_dm['error'], fmt='o', color='black', label='BAO観測データ')
ax2.plot(z_plot_range, DM_dirt_axis, color="red", label=r"DIRTモデル 軸方向")
ax2.plot(z_plot_range, DM_dirt_equator, color="green", linestyle='--', label=r"DIRTモデル 赤道方向")
ax2.plot(z_plot_range, DM_lcdm_line, color="gray", linestyle=':', label=r"標準ΛCDMモデル")
ax2.set_title(r"$D_M(z)/r_d$ の比較", fontproperties=font_prop)
ax2.set_xlabel("赤方偏移 z", fontproperties=font_prop)
ax2.set_ylabel(r"$D_M/r_d$", fontproperties=font_prop)
ax2.legend(prop=font_prop)
ax2.grid(True)

plt.suptitle("BAO観測データとDIRT Quadrupoleモデルの比較", fontproperties=font_prop, fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# ===== ステップ5: χ² (カイ二乗) による定量的評価 =====
chi2_dirt = np.sum(((bao_df['value'] - bao_df['theory_dirt']) / bao_df['error'])**2)
chi2_lcdm = np.sum(((bao_df['value'] - bao_df['theory_lcdm']) / bao_df['error'])**2)

print("\n--- カイ二乗 (χ²) 評価 ---")
print(f"DIRT Quadrupoleモデル の χ² = {chi2_dirt:.2f}")
print(f"標準ΛCDMモデル の χ² = {chi2_lcdm:.2f}")
if chi2_dirt < chi2_lcdm:
    print("→ DIRTモデルの方が、このBAOデータ点をより良く説明しています。")
else:
    print("→ 標準ΛCDMモデルの方が、このBAOデータ点をより良く説明しています。")

In [ ]:
# ===== ステップ0: 必要なライブラリのインストールとインポート =====
!pip install emcee corner tqdm

In [ ]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from scipy.integrate import quad
from astropy.coordinates import SkyCoord
from astropy import units as u
import emcee
import corner
from tqdm import tqdm

# 日本語フォント設定
!apt-get -y install fonts-noto-cjk
font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams["font.family"] = font_prop.get_name()
plt.rcParams["axes.unicode_minus"] = False


# ===== ステップ1: 全データの準備 =====

# --- SNe Ia データの準備 ---
sne_file_path = '/content/Pantheon+SH0ES.dat'
pantheon_data = pd.read_csv(sne_file_path, sep='\s+', comment='#')
# 角度計算
l_d = 264.02 * u.deg
b_d = 48.25 * u.deg
d_coord_galactic = SkyCoord(l=l_d, b=b_d, frame='galactic')
d_vector = d_coord_galactic.cartesian.xyz.value
sn_coords_icrs = SkyCoord(ra=pantheon_data['RA'], dec=pantheon_data['DEC'], unit='deg', frame='icrs')
sn_coords_galactic = sn_coords_icrs.galactic
n_vectors_sn = sn_coords_galactic.cartesian.xyz.value.T
pantheon_data['cos_theta'] = np.dot(n_vectors_sn, d_vector)

# --- BAO データの準備 ---
bao_data_list = [
    [0.70, 'DH', 180.0, 15.0, 19.33, 0.57], [0.70, 'DM', 180.0, 15.0, 17.86, 0.33],
    [1.48, 'DH', 195.0, 25.0, 13.23, 0.47], [1.48, 'DM', 195.0, 25.0, 30.21, 0.79],
    [0.50, 'DH', 190.0, 40.0, 23.39, 0.70], [0.50, 'DM', 190.0, 40.0, 14.05, 0.17],
    [0.90, 'DH', 210.0, 35.0, 17.58, 0.38], [0.90, 'DM', 210.0, 35.0, 22.01, 0.24],
    [1.60, 'DH', 170.0, 10.0, 12.39, 0.31], [1.60, 'DM', 170.0, 10.0, 32.11, 0.59]
]
bao_df = pd.DataFrame(bao_data_list, columns=['z', 'type', 'RA', 'DEC', 'value', 'error'])
# 角度計算
bao_coords_icrs = SkyCoord(ra=bao_df['RA'], dec=bao_df['DEC'], unit='deg', frame='icrs')
bao_coords_galactic = bao_coords_icrs.galactic
n_vectors_bao = bao_coords_galactic.cartesian.xyz.value.T
bao_df['cos_theta'] = np.dot(n_vectors_bao, d_vector)


# ===== ステップ1.5: テスト実行のためにデータ数を限定 =====
# 【重要】まずはこの設定で実行してください
N_sne_test = 50
N_bao_test = 10 # BAOは全点使用
data_sne_to_use = pantheon_data.sample(N_sne_test, random_state=42)
data_bao_to_use = bao_df.head(N_bao_test)
print(f"【テスト実行】SNe {len(data_sne_to_use)}個 + BAO {len(data_bao_to_use)}個のデータでMCMCを実行します。")

# --- 本番実行の場合は、以下の行を有効化 ---
# data_sne_to_use = pantheon_data
# data_bao_to_use = bao_df
# print(f"【本番実行】SNe {len(data_sne_to_use)}個 + BAO {len(data_bao_to_use)}個の全データでMCMCを実行します。")


# ===== ステップ2: モデルと尤度関数の定義 =====

# --- DIRTモデル関数 (前回と同様) ---
c_kms = 299792.458
r_d_fiducial = 147.09
def p2(ct): return 0.5 * (3 * ct**2 - 1)
def hubble_parameter_dirt(z, cos_theta, H0, Omega_m, R0, Ra, A2):
    a = 1.0 / (1.0 + z)
    Omega_rad = 9.24e-5
    Omega_L = 1.0 - Omega_m - Omega_rad
    r_iso = R0 + Ra * (1 - a)
    r_lock = np.maximum(0, r_iso * (1 + A2 * p2(cos_theta)))
    h2 = (Omega_rad / a**4) + (Omega_m / a**3) * r_lock + Omega_L
    return H0 * np.sqrt(h2)

# --- 2つの尤度関数を定義 ---
def log_likelihood_sne(params, data):
    """SNe Ia データに対する対数尤度"""
    H0, Omega_m, R0, Ra, A2 = params
    z = data['zCMB'].values
    mu_obs = data['m_b_corr'].values
    mu_err = data['m_b_corr_err_DIAG'].values
    cos_theta = data['cos_theta'].values

    chi2_sne = 0
    for i in range(len(z)):
        integrand = lambda z_prime: 1.0 / hubble_parameter_dirt(z_prime, cos_theta[i], H0, Omega_m, R0, Ra, A2)
        chi, _ = quad(integrand, 0, z[i])
        DL = c_kms * (1 + z[i]) * chi
        if DL <= 0: return -np.inf
        mu_th = 5 * np.log10(DL) + 25
        chi2_sne += ((mu_obs[i] - mu_th) / mu_err[i])**2

    return -0.5 * chi2_sne

def log_likelihood_bao(params, data):
    """BAO データに対する対数尤度"""
    H0, Omega_m, R0, Ra, A2 = params
    chi2_bao = 0
    for index, row in data.iterrows():
        z, type, ct, val, err = row['z'], row['type'], row['cos_theta'], row['value'], row['error']
        if type == 'DH':
            th = (c_kms / hubble_parameter_dirt(z, ct, H0, Omega_m, R0, Ra, A2)) / r_d_fiducial
        elif type == 'DM':
            chi = comoving_distance_dirt(z, ct, H0, Omega_m, R0, Ra, A2) # この関数は中で積分します
            th = chi / r_d_fiducial
        chi2_bao += ((val - th) / err)**2

    return -0.5 * chi2_bao

def comoving_distance_dirt(z, cos_theta, H0, Omega_m, R0, Ra, A2):
    """log_likelihood_baoで使うためのヘルパー関数"""
    integrand = lambda z_prime: 1.0 / hubble_parameter_dirt(z_prime, cos_theta, H0, Omega_m, R0, Ra, A2)
    chi, _ = quad(integrand, 0, z)
    return c_kms * chi

# --- 事前確率と事後確率 (統合版) ---
def log_prior(params):
    H0, Omega_m, R0, Ra, A2 = params
    if 60.0 < H0 < 80.0 and 0.1 < Omega_m < 0.5 and \
       0.8 < R0 < 1.2 and -1.0 < Ra < 1.0 and -0.5 < A2 < 0.5:
        return 0.0
    return -np.inf

def log_probability_joint(params, data_sne, data_bao):
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    # SNeとBAOの尤度を合算する
    ll_sne = log_likelihood_sne(params, data_sne)
    ll_bao = log_likelihood_bao(params, data_bao)
    return lp + ll_sne + ll_bao


# ===== ステップ3: MCMCの実行 =====
initial_params = np.array([70.0, 0.3, 1.0, 0.0, 0.0]) # 標準値に近いところからスタート
ndim = len(initial_params)
nwalkers = 32
nsteps = 2000 # テストでも少し長めに
burn_in = 500

p0 = initial_params + 1e-3 * np.random.randn(nwalkers, ndim)

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability_joint, args=(data_sne_to_use, data_bao_to_use))

print("統合MCMCサンプリングを開始します...")
sampler.run_mcmc(p0, nsteps, progress=True)
print("統合MCMCサンプリングが完了しました。")


# ===== ステップ4: 結果の可視化 =====
flat_samples = sampler.get_chain(discard=burn_in, thin=15, flat=True)
labels = [r"$H_0$", r"$\Omega_m$", r"$R_0$", r"$R_a$", r"$A_2$"]

fig = corner.corner(flat_samples, labels=labels, truths=initial_params, quantiles=[0.16, 0.5, 0.84], show_titles=True, title_kwargs={"fontsize": 12})
plt.show()

for i in range(ndim):
    mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    print(f"{labels[i]} = {mcmc[1]:.3f} +{q[1]:.3f} -{q[0]:.3f}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from scipy.integrate import quad
from astropy.coordinates import SkyCoord
from astropy import units as u
import emcee
import corner
from tqdm import tqdm

# 日本語フォント設定
!apt-get -y install fonts-noto-cjk
font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams["font.family"] = font_prop.get_name()
plt.rcParams["axes.unicode_minus"] = False


# ===== ステップ1: 全データの準備 =====

# --- SNe Ia データの準備 ---
sne_file_path = '/content/Pantheon+SH0ES.dat'
pantheon_data = pd.read_csv(sne_file_path, sep='\s+', comment='#')
# 角度計算
l_d = 264.02 * u.deg
b_d = 48.25 * u.deg
d_coord_galactic = SkyCoord(l=l_d, b=b_d, frame='galactic')
d_vector = d_coord_galactic.cartesian.xyz.value
sn_coords_icrs = SkyCoord(ra=pantheon_data['RA'], dec=pantheon_data['DEC'], unit='deg', frame='icrs')
sn_coords_galactic = sn_coords_icrs.galactic
n_vectors_sn = sn_coords_galactic.cartesian.xyz.value.T
pantheon_data['cos_theta'] = np.dot(n_vectors_sn, d_vector)

# --- BAO データの準備 ---
bao_data_list = [
    [0.70, 'DH', 180.0, 15.0, 19.33, 0.57], [0.70, 'DM', 180.0, 15.0, 17.86, 0.33],
    [1.48, 'DH', 195.0, 25.0, 13.23, 0.47], [1.48, 'DM', 195.0, 25.0, 30.21, 0.79],
    [0.50, 'DH', 190.0, 40.0, 23.39, 0.70], [0.50, 'DM', 190.0, 40.0, 14.05, 0.17],
    [0.90, 'DH', 210.0, 35.0, 17.58, 0.38], [0.90, 'DM', 210.0, 35.0, 22.01, 0.24],
    [1.60, 'DH', 170.0, 10.0, 12.39, 0.31], [1.60, 'DM', 170.0, 10.0, 32.11, 0.59]
]
bao_df = pd.DataFrame(bao_data_list, columns=['z', 'type', 'RA', 'DEC', 'value', 'error'])
# 角度計算
bao_coords_icrs = SkyCoord(ra=bao_df['RA'], dec=bao_df['DEC'], unit='deg', frame='icrs')
bao_coords_galactic = bao_coords_icrs.galactic
n_vectors_bao = bao_coords_galactic.cartesian.xyz.value.T
bao_df['cos_theta'] = np.dot(n_vectors_bao, d_vector)


# ===== ステップ1.5: テスト実行のためにデータ数を限定 =====
# 【重要】まずはこの設定で実行してください
# N_sne_test = 50
# N_bao_test = 10 # BAOは全点使用
# data_sne_to_use = pantheon_data.sample(N_sne_test, random_state=42)
# data_bao_to_use = bao_df.head(N_bao_test)
# print(f"【テスト実行】SNe {len(data_sne_to_use)}個 + BAO {len(data_bao_to_use)}個のデータでMCMCを実行します。")

# --- 本番実行の場合は、以下の行を有効化 ---
data_sne_to_use = pantheon_data
data_bao_to_use = bao_df
print(f"【本番実行】SNe {len(data_sne_to_use)}個 + BAO {len(data_bao_to_use)}個の全データでMCMCを実行します。")


# ===== ステップ2: モデルと尤度関数の定義 =====

# --- DIRTモデル関数 (前回と同様) ---
c_kms = 299792.458
r_d_fiducial = 147.09
def p2(ct): return 0.5 * (3 * ct**2 - 1)
def hubble_parameter_dirt(z, cos_theta, H0, Omega_m, R0, Ra, A2):
    a = 1.0 / (1.0 + z)
    Omega_rad = 9.24e-5
    Omega_L = 1.0 - Omega_m - Omega_rad
    r_iso = R0 + Ra * (1 - a)
    r_lock = np.maximum(0, r_iso * (1 + A2 * p2(cos_theta)))
    h2 = (Omega_rad / a**4) + (Omega_m / a**3) * r_lock + Omega_L
    return H0 * np.sqrt(h2)

# --- 2つの尤度関数を定義 ---
def log_likelihood_sne(params, data):
    """SNe Ia データに対する対数尤度"""
    H0, Omega_m, R0, Ra, A2 = params
    z = data['zCMB'].values
    mu_obs = data['m_b_corr'].values
    mu_err = data['m_b_corr_err_DIAG'].values
    cos_theta = data['cos_theta'].values

    chi2_sne = 0
    for i in range(len(z)):
        integrand = lambda z_prime: 1.0 / hubble_parameter_dirt(z_prime, cos_theta[i], H0, Omega_m, R0, Ra, A2)
        chi, _ = quad(integrand, 0, z[i])
        DL = c_kms * (1 + z[i]) * chi
        if DL <= 0: return -np.inf
        mu_th = 5 * np.log10(DL) + 25
        chi2_sne += ((mu_obs[i] - mu_th) / mu_err[i])**2

    return -0.5 * chi2_sne

def log_likelihood_bao(params, data):
    """BAO データに対する対数尤度"""
    H0, Omega_m, R0, Ra, A2 = params
    chi2_bao = 0
    for index, row in data.iterrows():
        z, type, ct, val, err = row['z'], row['type'], row['cos_theta'], row['value'], row['error']
        if type == 'DH':
            th = (c_kms / hubble_parameter_dirt(z, ct, H0, Omega_m, R0, Ra, A2)) / r_d_fiducial
        elif type == 'DM':
            chi = comoving_distance_dirt(z, ct, H0, Omega_m, R0, Ra, A2) # この関数は中で積分します
            th = chi / r_d_fiducial
        chi2_bao += ((val - th) / err)**2

    return -0.5 * chi2_bao

def comoving_distance_dirt(z, cos_theta, H0, Omega_m, R0, Ra, A2):
    """log_likelihood_baoで使うためのヘルパー関数"""
    integrand = lambda z_prime: 1.0 / hubble_parameter_dirt(z_prime, cos_theta, H0, Omega_m, R0, Ra, A2)
    chi, _ = quad(integrand, 0, z)
    return c_kms * chi

# --- 事前確率と事後確率 (統合版) ---
def log_prior(params):
    H0, Omega_m, R0, Ra, A2 = params
    if 60.0 < H0 < 80.0 and 0.1 < Omega_m < 0.5 and \
       0.8 < R0 < 1.2 and -1.0 < Ra < 1.0 and -0.5 < A2 < 0.5:
        return 0.0
    return -np.inf

def log_probability_joint(params, data_sne, data_bao):
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    # SNeとBAOの尤度を合算する
    ll_sne = log_likelihood_sne(params, data_sne)
    ll_bao = log_likelihood_bao(params, data_bao)
    return lp + ll_sne + ll_bao


# ===== ステップ3: MCMCの実行 =====
initial_params = np.array([70.0, 0.3, 1.0, 0.0, 0.0]) # 標準値に近いところからスタート
ndim = len(initial_params)
nwalkers = 32
nsteps = 2000 # テストでも少し長めに
burn_in = 500

p0 = initial_params + 1e-3 * np.random.randn(nwalkers, ndim)

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability_joint, args=(data_sne_to_use, data_bao_to_use))

print("統合MCMCサンプリングを開始します...")
sampler.run_mcmc(p0, nsteps, progress=True)
print("統合MCMCサンプリングが完了しました。")


# ===== ステップ4: 結果の可視化 =====
flat_samples = sampler.get_chain(discard=burn_in, thin=15, flat=True)
labels = [r"$H_0$", r"$\Omega_m$", r"$R_0$", r"$R_a$", r"$A_2$"]

fig = corner.corner(flat_samples, labels=labels, truths=initial_params, quantiles=[0.16, 0.5, 0.84], show_titles=True, title_kwargs={"fontsize": 12})
plt.show()

for i in range(ndim):
    mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    print(f"{labels[i]} = {mcmc[1]:.3f} +{q[1]:.3f} -{q[0]:.3f}")